In [1]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()

model = "qwen/qwen3-32b"

llm = ChatGroq(model = model, temperature = 0.0)

When we talk about agents, a significant part of an "agent" is simple code logic,
iteratively rerunning LLM calls and processing their output. The exact logic varies
significantly, but one well-known example is the **ReAct** agent.

![ReAct process](https://www.aurelio.ai/_next/image?url=%2Fimages%2Fposts%2Fai-agents%2Fai-agents-00.png&w=640&q=75)

**Re**ason + **Act**ion (ReAct) agents use iterative _reasoning_ and _action_ steps to
incorporate chain-of-thought and tool-use into their execution. During the _reasoning_
step, the LLM generates the steps to take to answer the query. Next, the LLM generates
the _action_ input, which our code logic parses into a tool call.

![Agentic graph of ReAct](https://www.aurelio.ai/_next/image?url=%2Fimages%2Fposts%2Fai-agents%2Fai-agents-01.png&w=640&q=75)

Following our action step, we get an observation from the tool call. Then, we feed the
observation back into the agent executor logic for a final answer or further reasoning
and action steps.

The agent and agent executor we will be building will follow this pattern.

In [2]:
# Tools.
from langchain_core.tools import tool

@tool
def add(x: float, y:float) -> float:
    """Add x and y"""

    return x+y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract x from y"""

    return y-x

@tool
def multiply(x: float, y: float) -> float:
    """Multiply x and y"""

    return x*y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise x to the power of y"""

    return x**y

In [3]:
add

StructuredTool(name='add', description='Add x and y', args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x000001826F945430>)

In [4]:
print(f"{add.name=}\n{add.description=}")

add.name='add'
add.description='Add x and y'


In [6]:
exponentiate.args_schema.model_json_schema()

{'description': 'Raise x to the power of y',
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'exponentiate',
 'type': 'object'}

In [8]:
import json

llm_output_string = "{\"x\" : 5, \"y\" : 2}"
llm_output_dict = json.loads(llm_output_string)
llm_output_dict

{'x': 5, 'y': 2}

In [9]:
exponentiate.func(**llm_output_dict)

25

In [10]:
subtract.func(**llm_output_dict)

-3

We need this agent to remember previous interactions within the conversation. To do that, we will use the `ChatPromptTemplate` with a system message, a placeholder for our chat history, a placeholder for the user query, and finally a placeholder for the agent scratchpad.

The agent scratchpad is where the agent writes its notes as it works through multiple internal thought and tool-use steps to produce a final output for the user. This scratchpad is a list of messages with alternating roles of `ai` (for the tool call) and `tool` (for the tool execution output). Both message types require a `tool_call_id` field which is used to link the respective AI and tool messages - this can be required when we many tool calls happening in parallel.

In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You're a helpful assistant. When answering a user's question "
        "you should first use one of the tools provided. After using a "
        "tool the tool output will be provided in the "
        "'scratchpad' below. If you have an answer in the "
        "scratchpad you should not use any more tools and "
        "instead answer directly to the user."
    )),
    MessagesPlaceholder(variable_name = "chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name = "agent_scratchpad")
])

In [21]:
from langchain_core.runnables.base import RunnableSerializable

tools = [add, subtract, multiply, exponentiate]

# define the agent runnable
agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history" : lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice = "any")
)

In [22]:
tool_call = agent.invoke(
    {
        "input" : "What is 10 times 10", "chat_history" : []
    }
)

tool_call

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking "What is 10 times 10". Let me see. The available functions are add, subtract, multiply, and exponentiate. The question is about multiplication, so the correct function here would be multiply. The parameters needed are x and y, both numbers. Here, x is 10 and y is 10. So I should call the multiply function with these values. Let me double-check to make sure I\'m not mixing up any operations. Yeah, multiply is the right choice. The result should be 100. I don\'t need to use any other functions here.\n', 'tool_calls': [{'id': 'bvqn7tj40', 'function': {'arguments': '{"x":10,"y":10}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 160, 'prompt_tokens': 429, 'total_tokens': 589, 'completion_time': 0.355998387, 'prompt_time': 0.036793107, 'queue_time': 0.047199173, 'total_time': 0.392791494}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf9

In [23]:
tool_call.tool_calls

[{'name': 'multiply',
  'args': {'x': 10, 'y': 10},
  'id': 'bvqn7tj40',
  'type': 'tool_call'}]

From here, we have the tool `name` that our LLM wants to use and the `args` that it
wants to pass to that tool. We can see that the tool `add` is being used with the
arguments `x=10` and `y=10`. The `agent.invoke` method has _not_ executed the tool
function; we need to write that part of the agent code ourselves.

Executing the tool code requires two steps:

1. Map the tool `name` to the tool function.

2. Execute the tool function with the generated `args`.

In [26]:
# create tool name to function mapping

name2tool = {tool.name : tool.func for tool in tools}
name2tool

{'add': <function __main__.add(x: float, y: float) -> float>,
 'subtract': <function __main__.subtract(x: float, y: float) -> float>,
 'multiply': <function __main__.multiply(x: float, y: float) -> float>,
 'exponentiate': <function __main__.exponentiate(x: float, y: float) -> float>}

In [25]:
tool_exec_content = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)

tool_exec_content

100

In [32]:
from langchain_core.messages import ToolMessage

tool_exec = ToolMessage(
    content = f"The {tool_call.tool_calls[0]['name']} tool returned {tool_exec_content}",
    tool_call_id = tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input" : "What is 10 minus 10",
    "chat_history" : [],
    "agent_scratchpad" : [tool_call, tool_exec]
})

out

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user asked "What is 10 minus 10?" Let me see. I need to figure out which tool to use here. The available functions are add, subtract, multiply, and exponentiate.\n\nThe user is asking for subtraction, so the correct function should be subtract. But wait, in the previous interaction, the assistant used the multiply function by mistake. The user\'s current question is the same as before, but maybe they want the correct answer this time. \n\nLet me check the parameters. The subtract function requires x and y. The question is 10 minus 10, so x is 10 and y is 10. The correct tool is subtract, not multiply. The previous tool call was wrong, so I should correct that. \n\nI need to make sure to call the subtract function with x=10 and y=10. The result should be 0. Let me structure the tool call properly. The function name is subtract, and the arguments are x:10, y:10. \n\nNo other tools are needed here. Just the subtract 

Despite having the answer in our `agent_scratchpad`, the LLM still tries to use the tool
_again_. This behaviour happens because we bonded the tools to the LLM with
`tool_choice="any"`. When we set `tool_choice` to `"any"` or `"required"`, we tell the
LLM that it _MUST_ use a tool, i.e., it cannot provide a final answer.

There's two options to fix this:

1. Set `tool_choice="auto"` to tell the LLM that it can choose to use a tool or provide
a final answer.

2. Create a `final_answer` tool - we'll explain this shortly.

First, let's try option **1**:

In [37]:
agent: RunnableSerializable = (
    {
        "input" : lambda x: x["input"],
        "chat_history" : lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice = "auto")

)

In [38]:
tool_call= agent.invoke(
    {
        "input" : "What is 10 minus 10",
        "chat_history" : []
    }
)

tool_call

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking "What is 10 minus 10?" Let me see. I need to figure out which tool to use here. The available functions are add, subtract, multiply, and exponentiate.\n\nThe question is about subtraction, so the subtract function is the right choice. The parameters for subtract are x and y, where x is the number being subtracted from, and y is the number subtracted. Wait, actually, the function description says "Subtract x from y", which might mean that the function subtracts x from y, making the result y - x. So in this case, the user is asking 10 minus 10, which would be y - x where y is 10 and x is 10. So I need to set x to 10 and y to 10 in the subtract function. Let me double-check that. If the function is called subtract and the parameters are x and y, but the description says "Subtract x from y", then the operation is y - x. So for 10 minus 10, y is 10 and x is 10. Therefore, the arguments should be x: 10, y

In [42]:
tool_output = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)

tool_exec = ToolMessage(
    content = f"The {tool_call.tool_calls[0]['name']} tool returned {tool_output}",
    tool_call_id = tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input" : "What is 10 minus 10",
    "chat_history" : [],
    "agent_scratchpad" : [tool_call, tool_exec]
})

out

AIMessage(content='10 minus 10 equals 0.', additional_kwargs={'reasoning_content': 'Okay, the user asked "What is 10 minus 10?" So I need to subtract 10 from 10. Let me check the available tools. There\'s a subtract function that takes x and y. I\'ll call that with x=10 and y=10. The result from the tool was 0. So the answer is 0. I should just state that clearly.\n'}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 475, 'total_tokens': 575, 'completion_time': 0.153876734, 'prompt_time': 0.03037699, 'queue_time': 0.053233138, 'total_time': 0.184253724}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--92599875-e101-4219-9880-4fc58548631c-0', usage_metadata={'input_tokens': 475, 'output_tokens': 100, 'total_tokens': 575})

option 2 - final answer tool

In [51]:
@tool
def final_answer(answer: str, tools_used: list[str]) -> str:
    """Use this tool to provide a final answer to the user.
    The answer should be in natural language as this will be provided
    to the user directly. The tools_used must include a list of tool
    names that were used within the `scratchpad`.
    """
    return {"answer": answer, "tools_used": tools_used}

In [52]:
tools = [final_answer, add, subtract, multiply, exponentiate]

# we need to update our name2tool mapping too
name2tool = {tool.name: tool.func for tool in tools}

agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")  # we're forcing tool use again
)

In [53]:
tool_call = agent.invoke({"input": "What is 10 + 10", "chat_history": []})
tool_call.tool_calls

[{'name': 'add',
  'args': {'x': 10, 'y': 10},
  'id': 'j7ahk9ngm',
  'type': 'tool_call'}]

In [54]:
tool_out = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)

tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_out}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input": "What is 10 + 10",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_exec]
})
out

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user asked "What is 10 + 10". I used the add function with x=10 and y=10. The result was 20. Now I need to present the final answer. Since the tool already gave the correct result, I just need to state it clearly. Make sure to mention the tools used, which is the add function here. No other tools are needed. Just confirm the calculation and provide the answer in a natural sentence.\n', 'tool_calls': [{'id': 'dhkqwavxq', 'function': {'arguments': '{"answer":"The result of 10 + 10 is 20.","tools_used":["add"]}', 'name': 'final_answer'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 159, 'prompt_tokens': 603, 'total_tokens': 762, 'completion_time': 0.326689163, 'prompt_time': 0.08433144, 'queue_time': 0.049523569, 'total_time': 0.411020603}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': 'fp_5cf921caa2', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': No

In [55]:
out.tool_calls

[{'name': 'final_answer',
  'args': {'answer': 'The result of 10 + 10 is 20.', 'tools_used': ['add']},
  'id': 'dhkqwavxq',
  'type': 'tool_call'}]

In [56]:
out.tool_calls[0]["args"]

{'answer': 'The result of 10 + 10 is 20.', 'tools_used': ['add']}